# Train model

In [1]:
batch_size=32
pickle_file='test_dataset_cluster_labeled.pickle'
limit_seq_len=1000

In [2]:
from dataset import DwgDataset
dwg_dataset = DwgDataset(pickle_file=pickle_file, batch_size=batch_size, limit_seq_len=limit_seq_len)

In [3]:
lr=0.001
epochs=500
train_verbose=True

In [4]:
from sketch_rnn import Trainer
from IPython.display import clear_output
from plot_graphics import plot_history

trainer = Trainer(
    dwg_dataset, 
    lr=lr, 
    train_verbose=train_verbose)

train_kl_losses = []
train_rl_losses = []
val_kl_losses  = []
val_rl_losses  = []

for epoch in range(epochs):
    #clear_output(wait=True)
    
    #plot_history(train_rl_losses, train_kl_losses, val_rl_losses, val_kl_losses)
    test_rl, test_kl, val_rl, val_kl = trainer.train_epoch(epoch)
    
    train_kl_losses.append(test_kl)
    train_rl_losses.append(test_rl)
    val_kl_losses.append(val_kl)
    val_rl_losses.append(val_rl)

# Calculate test accuracy
test_kl, test_rl = trainer.CalculateLoaderAccuracy(trainer.test_loader)
print('Test losses rl:{:1.4f} kl:{:1.4f}'.format(test_rl, test_kl))    

Could not import drawSvg


C:\Users\mihanick.MCAD\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


  [   0-   0 @   8.6 sec] RLoss: 2.52813 KL Loss: 0.0037
  [   0-   1 @  15.6 sec] RLoss: 2.54091 KL Loss: 0.0033
Epoch [0 @ 16.2] validation losses rl:1.2357 kl:0.0024
  [   1-   0 @   0.9 sec] RLoss: 2.46937 KL Loss: 0.0046
  [   1-   1 @   1.7 sec] RLoss: 2.29330 KL Loss: 0.0074
Epoch [1 @  2.0] validation losses rl:1.1928 kl:0.0072
  [   2-   0 @   0.9 sec] RLoss: 2.33552 KL Loss: 0.0134
  [   2-   1 @   1.7 sec] RLoss: 2.00008 KL Loss: 0.0239
Epoch [2 @  1.9] validation losses rl:0.9425 kl:0.0254
  [   3-   0 @   0.9 sec] RLoss: 1.58711 KL Loss: 0.0478
  [   3-   1 @   1.7 sec] RLoss: 0.95387 KL Loss: 0.0971
Epoch [3 @  1.9] validation losses rl:0.3450 kl:0.1262
  [   4-   0 @   0.9 sec] RLoss: -0.01538 KL Loss: 0.2375
  [   4-   1 @   1.7 sec] RLoss: 0.74995 KL Loss: 0.1694
Epoch [4 @  1.9] validation losses rl:0.3098 kl:0.0606
  [   5-   0 @   0.9 sec] RLoss: -0.18835 KL Loss: 0.1139
  [   5-   1 @   1.7 sec] RLoss: 0.38412 KL Loss: 0.0735
Epoch [5 @  1.9] validation losses rl:1

KeyboardInterrupt: 

# Evaluate model

In [ ]:
stroke_features = dwg_dataset.entities.stroke_features
max_seq_length = dwg_dataset.entities.max_seq_length

enc_hidden_size    = 256
dec_hidden_size    = 512

d_z=128
n_distributions    = 20

from model import DecoderRNN, EncoderRNN
encoder = EncoderRNN(d_z, enc_hidden_size, stroke_features=stroke_features)
decoder = DecoderRNN(d_z, dec_hidden_size, n_distributions, stroke_features=stroke_features)
encoder.to(device)
decoder.to(device)

# https://pytorch.org/tutorials/beginner/saving_loading_models.html
encoder.load_state_dict(torch.load('DimEncoder.model', map_location=device))
decoder.load_state_dict(torch.load('DimDecoder.model', map_location=device))

decoder.eval()
encoder.eval()

In [ ]:
import drawSvg as draw
from plot_graphics  import images_from_batch

In [ ]:
from IPython.display import Image
from IPython.display import clear_output

In [ ]:
for i, batch in enumerate(dwg_dataset.train_loader):
    
    data = batch[0].to(device).transpose(0, 1)
    mask = batch[1].to(device).transpose(0, 1)
    
    for i in images_from_batch(data):
        display(i)
    #sampler = Sampler(encoder, decoder)
    #seq = sampler.sample(data, 0.4)
    #print(seq)
    break